# Notebook that reads in typhoon data and computes 6 hr and 24 hr max in mm/hr 

In [ ]:
%load_ext jupyter_black

import pandas as pd
import os
from pathlib import Path
import math
import datetime

# input
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/03_rainfall/input"
)

# outputs
processed_output_dir = (
    input_dir / "gpm_data/rainfall_data/output_hhr_processed/"
)

output_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/03_rainfall/output"
)
typhoon_metadata = pd.read_csv(input_dir / "metadata_typhoons.csv")

In [ ]:
# loading csv files
typ_list = [
    x for x in os.listdir(processed_output_dir) if x.endswith("_gridstats.csv")
]
# typ_file = pd.read_csv(processed_output_dir + typ_list[0])
# typ_file

This extracts the 6 hr and 24 hr max for each hour for both the mean and max columns.

In [ ]:
# To make sure the dates can be converted to date type
typhoon_metadata["startdate"] = [
    str_col.replace("/", "-") for str_col in typhoon_metadata["startdate"]
]
typhoon_metadata["enddate"] = [
    str_col.replace("/", "-") for str_col in typhoon_metadata["enddate"]
]
typhoon_metadata["landfalldate"] = [
    str_col.replace("/", "-") for str_col in typhoon_metadata["landfalldate"]
]

typhoon_metadata["landfall_date_time"] = (
    typhoon_metadata["landfalldate"] + "-" + typhoon_metadata["landfall_time"]
)

typhoons = list(typhoon_metadata["typhoon"].values)
#%% Processing the data into an excel sheet
time_frame_24 = 48  # in half hours
time_frame_6 = 12  # in half hours
mov_window = 12  # in half hours
before_landfall_h = 72  # how many hours before landfall to include
after_landfall_h = 72  # how many hours before landfall to include

In [ ]:
# looping over all typhoons
for stats in ["mean", "max"]:
    df_rainfall_final = pd.DataFrame(
        columns=["typhoon", "id", "Centroid", "rainfall_Total"]
    )
    for typ in typhoons:
        print(typ)
        # Getting typhoon info
        df_info = typhoon_metadata[typhoon_metadata["typhoon"] == typ]
        landfall = df_info["landfall_date_time"].values[0]
        landfall = datetime.datetime.strptime(landfall, "%d-%m-%Y-%H:%M:%S")
        # End date is landfall date
        # Start date is 72 hours before landfall date
        # end_date = landfall
        end_date = landfall + datetime.timedelta(
            hours=after_landfall_h
        )  # landfall
        # start_date = end_date - datetime.timedelta(hours=before_landfall_h)
        start_date = landfall - datetime.timedelta(hours=before_landfall_h)
        # Loading the data
        df_rainfall = pd.read_csv(
            processed_output_dir / str(typ + "_gridstats_" + stats + ".csv")
        )
        # Convert column names to date format
        for col in df_rainfall.columns[2:]:
            date_format = datetime.datetime.strptime(col, "%Y%m%d_%H:%M:%S")
            df_rainfall = df_rainfall.rename(columns={col: date_format})

        df_mean_rainfall = pd.DataFrame(
            {"id": df_rainfall["id"], "Centroid": df_rainfall["Centroid"]}
        )
        available_dates_t = [
            date
            for date in df_rainfall.columns[2:]
            if (date >= start_date) & (date < end_date)
        ]
        #####################################
        df_mean_rainfall["rainfall_max_6h"] = (
            df_rainfall.iloc[:, 2:]
            .rolling(time_frame_6, axis=1)
            .mean()
            .max(axis=1)
        )

        df_mean_rainfall["rainfall_max_24h"] = (
            df_rainfall.iloc[:, 2:]
            .rolling(time_frame_24, axis=1)
            .mean()
            .max(axis=1)
        )

        df_mean_rainfall["rainfall_Total"] = 0.5 * df_rainfall[
            available_dates_t
        ].sum(axis=1)

        df_rainfall_single = df_mean_rainfall[
            [
                "id",
                "Centroid",
                "rainfall_max_6h",
                "rainfall_max_24h",
                "rainfall_Total",
            ]
        ]
        df_rainfall_single["typhoon"] = typ
        df_rainfall_final = pd.concat([df_rainfall_final, df_rainfall_single])
    df_rainfall_final.to_csv(
        output_dir / str("rainfall_data_rw_" + stats + ".csv"), index=False
    )